In [306]:
#importing data management, data visualisation libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

In [307]:
data= pd.read_csv('KL_no_null_no_duplicates_data.csv')
data

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3,2,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4,2,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
2,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3,2,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished
3,"Taman Tun Dr Ismail, Kuala Lumpur","RM 5,350,000",4+2,5,4,Bungalow,Land area : 7200 sq. ft.,Partly Furnished
4,"Taman Tun Dr Ismail, Kuala Lumpur","RM 2,600,000",5,4,4,Semi-detached House,Land area : 3600 sq. ft.,Partly Furnished
...,...,...,...,...,...,...,...,...
29027,"Seputeh, Kuala Lumpur","RM 750,000",3,2,1,Condominium,Built-up : 915 sq. ft.,Partly Furnished
29028,"KL Sentral, Kuala Lumpur","RM 1,400,000",3+1,3,2,Condominium (Corner),Land area : 1544 sq. ft.,Fully Furnished
29029,"KL Eco City, Kuala Lumpur","RM 880,000",1,1,1,Condominium (Corner),Built-up : 650 sq. ft.,Partly Furnished
29030,"Sri Hartamas, Kuala Lumpur","RM 2,700,000",4+2,6,3,Condominium (Corner),"Built-up : 3,973 sq. ft.",Partly Furnished


In [308]:
df=data.copy()

In [309]:
# Assuming 'df' is your DataFrame and 'location' is the column containing location names
df['Location'] = df['Location'].str.replace(', Kuala Lumpur', '')

# Remove leading and trailing whitespaces
df['Location'] = df['Location'].str.strip()
df

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,KLCC,"RM 1,250,000",2+1,3,2,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,Dutamas,"RM 1,030,000",3,4,2,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
2,Bukit Jalil,"RM 900,000",4+1,3,2,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished
3,Taman Tun Dr Ismail,"RM 5,350,000",4+2,5,4,Bungalow,Land area : 7200 sq. ft.,Partly Furnished
4,Taman Tun Dr Ismail,"RM 2,600,000",5,4,4,Semi-detached House,Land area : 3600 sq. ft.,Partly Furnished
...,...,...,...,...,...,...,...,...
29027,Seputeh,"RM 750,000",3,2,1,Condominium,Built-up : 915 sq. ft.,Partly Furnished
29028,KL Sentral,"RM 1,400,000",3+1,3,2,Condominium (Corner),Land area : 1544 sq. ft.,Fully Furnished
29029,KL Eco City,"RM 880,000",1,1,1,Condominium (Corner),Built-up : 650 sq. ft.,Partly Furnished
29030,Sri Hartamas,"RM 2,700,000",4+2,6,3,Condominium (Corner),"Built-up : 3,973 sq. ft.",Partly Furnished


In [310]:
df['Location'].nunique()

79

In [311]:
df['Location'].value_counts()

Location
Mont Kiara                           3324
KLCC                                 2812
Cheras                               2365
Jalan Klang Lama (Old Klang Road)    1804
Bukit Jalil                          1611
                                     ... 
Jalan U-Thant                           1
Rawang                                  1
duta Nusantara                          1
Casa Rimba                              1
Canary Residence                        1
Name: count, Length: 79, dtype: int64

In [312]:
# Find locations with counts less than 10
locations_to_replace = df['Location'].value_counts()[df['Location'].value_counts() < 10].index.tolist()


In [313]:
# Without inplace=True, assign the result back to the DataFrame
df['Location'] = df['Location'].replace(locations_to_replace, 'Other')
df

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,KLCC,"RM 1,250,000",2+1,3,2,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,Dutamas,"RM 1,030,000",3,4,2,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
2,Bukit Jalil,"RM 900,000",4+1,3,2,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished
3,Taman Tun Dr Ismail,"RM 5,350,000",4+2,5,4,Bungalow,Land area : 7200 sq. ft.,Partly Furnished
4,Taman Tun Dr Ismail,"RM 2,600,000",5,4,4,Semi-detached House,Land area : 3600 sq. ft.,Partly Furnished
...,...,...,...,...,...,...,...,...
29027,Seputeh,"RM 750,000",3,2,1,Condominium,Built-up : 915 sq. ft.,Partly Furnished
29028,KL Sentral,"RM 1,400,000",3+1,3,2,Condominium (Corner),Land area : 1544 sq. ft.,Fully Furnished
29029,KL Eco City,"RM 880,000",1,1,1,Condominium (Corner),Built-up : 650 sq. ft.,Partly Furnished
29030,Sri Hartamas,"RM 2,700,000",4+2,6,3,Condominium (Corner),"Built-up : 3,973 sq. ft.",Partly Furnished


In [314]:
df['Size']

0        Built-up : 1,335 sq. ft.
1        Built-up : 1,875 sq. ft.
2        Built-up : 1,513 sq. ft.
3        Land area : 7200 sq. ft.
4        Land area : 3600 sq. ft.
                   ...           
29027      Built-up : 915 sq. ft.
29028    Land area : 1544 sq. ft.
29029      Built-up : 650 sq. ft.
29030    Built-up : 3,973 sq. ft.
29031    Built-up : 1,313 sq. ft.
Name: Size, Length: 29032, dtype: object

In [315]:
# Extracted Type from Size column

import pandas as pd

# Assuming df is your DataFrame with the 'Size' column

# Extracting type from 'Size' column and removing it from the original column
def extract_type(x):
    if isinstance(x, str):
        if 'Built-up' in x:
            type_part = 'Built-up'
            size_part = x.replace('Built-up', '').strip()
        elif 'Land area' in x:
            type_part = 'Land area'
            size_part = x.replace('Land area', '').strip()
        else:
            type_part = None
            size_part = x.strip()
        return type_part, size_part
    else:
        return None, None

# Applying the extract_type function to create a new 'Type' column
df['Type'], df['Size'] = zip(*df['Size'].apply(extract_type))

# Displaying the result
print(df['Size'])


0        : 1,335 sq. ft.
1        : 1,875 sq. ft.
2        : 1,513 sq. ft.
3         : 7200 sq. ft.
4         : 3600 sq. ft.
              ...       
29027      : 915 sq. ft.
29028     : 1544 sq. ft.
29029      : 650 sq. ft.
29030    : 3,973 sq. ft.
29031    : 1,313 sq. ft.
Name: Size, Length: 29032, dtype: object


In [316]:
df['Type']

0         Built-up
1         Built-up
2         Built-up
3        Land area
4        Land area
           ...    
29027     Built-up
29028    Land area
29029     Built-up
29030     Built-up
29031     Built-up
Name: Type, Length: 29032, dtype: object

In [317]:
df['Type'].isna().sum()

0

In [318]:
df['Type'].value_counts()

Type
Built-up     22604
Land area     6428
Name: count, dtype: int64

In [319]:
# Remove dots ('.'), commas (','), and colons (':')

df['Size'] = df['Size'].str.replace('[.,:]', '', regex=True)
df['Size']

0         1335 sq ft
1         1875 sq ft
2         1513 sq ft
3         7200 sq ft
4         3600 sq ft
            ...     
29027      915 sq ft
29028     1544 sq ft
29029      650 sq ft
29030     3973 sq ft
29031     1313 sq ft
Name: Size, Length: 29032, dtype: object

In [320]:
df['Size'].isna().sum()

0

In [321]:
df['Size'].nunique()

3485

In [322]:
df['Size'].value_counts()

Size
 1100 sq ft     392
 950 sq ft      294
 1200 sq ft     265
 22x75 sq ft    261
 850 sq ft      235
               ... 
 1968 sq ft       1
 10355 sq ft      1
 13558 sq ft      1
 6135 sq ft       1
 2132 sq ft       1
Name: count, Length: 3485, dtype: int64

In [323]:
# Removing Spaces

import pandas as pd
import re

# Assuming df is your DataFrame with the 'Size' column

# Function to handle unary '-' without any numeric value and replace 'N/A' with 0
def handle_unary_na(size_str):
    if isinstance(size_str, str):
        # Remove spaces
        size_str = size_str.replace(' ', '')
        # Replace 'N/A' with 0
        size_str = size_str.replace('N/A', '0')
        # Handling unary '-' without numeric value
        size_str = re.sub(r'(?<!\d)-(?!\d)', '0', size_str)
        # Removing unary '+' signs if they exist
        size_str = size_str.replace('+', '')
        return size_str
    else:
        return size_str

# Applying the handle_unary_na function to the 'Size' column
df['Size'] = df['Size'].apply(handle_unary_na)

# Displaying the result
print(df[['Size']])


           Size
0      1335sqft
1      1875sqft
2      1513sqft
3      7200sqft
4      3600sqft
...         ...
29027   915sqft
29028  1544sqft
29029   650sqft
29030  3973sqft
29031  1313sqft

[29032 rows x 1 columns]


In [324]:
# Extracting Operators

import pandas as pd
import re

# Assuming df is your DataFrame with the 'Size' column

# Function to search for operators in the 'Size' column
def search_operators(size_str):
    if isinstance(size_str, str):
        # Regular expression pattern to match any non-alphanumeric or non-whitespace character
        pattern = r'[^\w\s]'  # Matches any character that is not alphanumeric or whitespace
        # Search for operators in the string
        matches = re.findall(pattern, size_str)
        # Return the matches found
        return ', '.join(matches) if matches else None
    else:
        return None



In [325]:
# Applying the search_operators function to the 'Size' column
matched_operators = df['Size'].apply(search_operators)

# Displaying the Size column values along with the matched operators
result_df = pd.DataFrame({'Size': df['Size'], 'Matched_Operators': matched_operators})
print(result_df.dropna())


                   Size Matched_Operators
241    646sf~1001sfsqft                 ~
254    850sf~1000sfsqft                 ~
259        850-1000sqft                 -
784        646~2928sqft                 ~
786      840~1193sfsqft                 ~
...                 ...               ...
26167       22'X85'sqft              ', '
26459       22'X75'sqft              ', '
26659       40'x75'sqft              ', '
27249    55&#215;80sqft           &, #, ;
28300         15*60sqft                 *

[71 rows x 2 columns]


In [326]:
# Making 0 if #, & or ; exists

def check_characters(s):
    if s is not None and ('#' in s or '&' in s or ';' in s or '\'' in s):
        return 0
    else:
        return s

# Apply the function to the 'Size' column
df['Size'] = df['Size'].apply(check_characters)
df['Size']

0        1335sqft
1        1875sqft
2        1513sqft
3        7200sqft
4        3600sqft
           ...   
29027     915sqft
29028    1544sqft
29029     650sqft
29030    3973sqft
29031    1313sqft
Name: Size, Length: 29032, dtype: object

In [327]:
df['Size']

0        1335sqft
1        1875sqft
2        1513sqft
3        7200sqft
4        3600sqft
           ...   
29027     915sqft
29028    1544sqft
29029     650sqft
29030    3973sqft
29031    1313sqft
Name: Size, Length: 29032, dtype: object

In [328]:
# Function to handle unary '-' and '~' without any numeric value and replace 'N/A' with 0

import re
def handle_unary_na(size_str):
    if isinstance(size_str, str):
        # Handling unary '-' and '~' without numeric value
        size_str = re.sub(r'(?<!\d)[-~](?!\d)', '0', size_str)
        # Removing unary '+' signs if they exist
        size_str = size_str.replace('+', '')
        return size_str
    else:
        return size_str

# Function to replace number ranges with their average
def replace_range_with_average(size_str):
    if isinstance(size_str, str):
        # Extracting numbers from the range
        numbers = re.findall(r'\d+', size_str)
        if numbers:
            # Converting numbers to integers
            numbers = [int(num) for num in numbers]
            # Calculating average of the numbers
            average = sum(numbers) / len(numbers)
            # Replacing the range with the average
            size_str = str(average)
        return size_str
    else:
        return size_str

# Function to reattach units
def reattach_units(size_str, original_size):
    if isinstance(original_size, str):
        return f"{size_str}{original_size[-6:]}"  # assuming units are at the end and in the format 'xxxxsqft'
    else:
        return size_str

# Save original sizes for reattaching units later
df['original_size'] = df['Size']

# Applying the functions to the 'Size' column
df['Size'] = df['Size'].apply(handle_unary_na)
df['Size'] = df['Size'].apply(replace_range_with_average)

# Retain units
df['Size'] = df.apply(lambda row: reattach_units(row['Size'], row['original_size']), axis=1)

# Displaying the result with units retained
print(df[['Size']])


               Size
0      1335.035sqft
1      1875.075sqft
2      1513.013sqft
3      7200.000sqft
4      3600.000sqft
...             ...
29027   915.015sqft
29028  1544.044sqft
29029   650.050sqft
29030  3973.073sqft
29031  1313.013sqft

[29032 rows x 1 columns]


In [329]:
# Treated x,X,*,/ to perform multiplication and + to do addition

import pandas as pd
import re

# Assuming df is your DataFrame with the 'Size' column

# Function to perform arithmetic operations if 'x', 'X', '*', '/', or '+' is found in the size string
def perform_arithmetic(size_str):
    if isinstance(size_str, str):
        # Regular expression to match arithmetic operations
        match = re.search(r'(\d+)\s*([xX*\/\+])\s*(\d+)\s*(\w*)', size_str)
        if match:
            # Extracting the numbers and the operator
            num1 = int(match.group(1))
            operator = match.group(2)
            num2 = int(match.group(3))
            # Performing the arithmetic operation
            if operator.lower() == 'x':
                result = num1 * num2
            elif operator == '*':
                result = num1 * num2
            elif operator == '/':
                result = num1 * num2  # Treat '/' as multiplication
            elif operator == '+':
                result = num1 + num2
            else:  # Ignore other operators
                return size_str

            unit = match.group(4)
            if unit:
                return f"{result} {unit}"
            else:
                return str(result)

        return size_str
    else:
        return size_str

# Applying the perform_arithmetic function to the 'Size' column
df['Size'] = df['Size'].apply(perform_arithmetic)

# Displaying the result
print(df[['Size']])


               Size
0      1335.035sqft
1      1875.075sqft
2      1513.013sqft
3      7200.000sqft
4      3600.000sqft
...             ...
29027   915.015sqft
29028  1544.044sqft
29029   650.050sqft
29030  3973.073sqft
29031  1313.013sqft

[29032 rows x 1 columns]


In [330]:
# print the unique sizes excluding 'sqft' and 0 values from the given list of rows.


df['Size'] = df['Size'].fillna(0)

# Filter out rows where 'Size' column does not contain 'sqft' or is 0
size_other_than_sqft_or_zero = df[(~df['Size'].astype(str).str.contains('sqft')) & (df['Size'] != 0)]

# Extract unique values
unique_sizes = size_other_than_sqft_or_zero['Size'].unique()

# Display unique values
print("Unique Sizes:")
for size in unique_sizes:
    print(size)


Unique Sizes:
1405.0405sqm
1707.0707sqm
1092.0092sqm
189.0189sqm
0 sqm
237.0237sqm
1111.0111sqm
6.06acres
1336.06acres
2906.06acres
1535.0535sqm
1569.0569sqm
1237.0237sqm
425 sqm
4150.00acres
70.070sqm
2625.0625sqm
1430.0430sqm
886.06acres
2723.0723sqm
2065.0065sqm
2131.0131sqm
980.0980sqm
987.0987sqm
1442.0442sqm
60.00acres
8000.0000sqm
2750.0750sqm
1299.09acres
57.07acres
1207.0207sqm
1663.0663sqm
1109.0109sqm
2200.0200sqm
3478.0478sqm
3667.0667sqm
279.0279sqm
1242.0242sqm
1301.0301sqm


In [331]:
# Function to replace 'Kuala', 'Lumpur', 'None', '00ur', or 'WilayahPersekutuan' with '0' in the size string
def replace_kl_none(size_str):
    if isinstance(size_str, str):
        # Replacing 'Kuala', 'Lumpur', '00ur', 'None', or 'WilayahPersekutuan' with '0' in the string
        return size_str.replace('Kuala', '0').replace('Lumpur', '0').replace('None', '0').replace('00ur', '0').replace('WilayahPersekutuan', '0').strip()
    elif size_str is None:
        return '0'
    else:
        return size_str

# Applying the replace_kl_none function to the 'Size' column
df['Size'] = df['Size'].apply(replace_kl_none)

# Displaying the result
print(df['Size'])


0        1335.035sqft
1        1875.075sqft
2        1513.013sqft
3        7200.000sqft
4        3600.000sqft
             ...     
29027     915.015sqft
29028    1544.044sqft
29029     650.050sqft
29030    3973.073sqft
29031    1313.013sqft
Name: Size, Length: 29032, dtype: object


In [332]:
# Conversion factors for different units to sqft

unit_conversion = {
    'sqm': 10.7639,   # 1 square meter = 10.7639 square feet
    'acres': 43560,   # 1 acre = 43560 square feet
    'hectare': 107639.1,   # 1 hectare = 107639.1 square feet
    'sf': 1,  # 1 square foot = 1 square foot
    'ft': 1  # 1 foot = 1 foot
}

# Function to convert sizes to sqft
def convert_to_sqft(size_str):
    if isinstance(size_str, str):
        if 'sqft' not in size_str and 'sf' not in size_str:
            if 'sqm' in size_str:
                return str(float(size_str.replace('sqm', '')) * unit_conversion['sqm']) + ' sqft'
            elif 'acres' in size_str:
                try:
                    return str(float(size_str.replace('acres', '')) * unit_conversion['acres']) + ' sqft'
                except ValueError:
                    return None
            elif 'hectare' in size_str or 'ectare' in size_str:
                try:
                    return str(float(size_str.replace('hectare', '').replace('ectare', '')) * unit_conversion['hectare']) + ' sqft'
                except ValueError:
                    return None
            elif 'ft' in size_str:
                try:
                    return str(float(size_str.replace('ft', '')) * unit_conversion['ft'] * unit_conversion['ft']) + ' sqft'
                except ValueError:
                    return None
            else:
                try:
                    return size_str + ' sqft'
                except ValueError:
                    return None
        else:
            try:
                return size_str.replace('sf', 'sqft')
            except ValueError:
                return None
    else:
        return size_str

# Applying the conversion function to the 'Size' column
df['Size'] = df['Size'].apply(convert_to_sqft)

# Displaying the result
print(df[['Size']])


               Size
0      1335.035sqft
1      1875.075sqft
2      1513.013sqft
3      7200.000sqft
4      3600.000sqft
...             ...
29027   915.015sqft
29028  1544.044sqft
29029   650.050sqft
29030  3973.073sqft
29031  1313.013sqft

[29032 rows x 1 columns]


In [333]:
# Remove 'sqft' and 'sf' from the 'Size' column

import pandas as pd

# Assuming you have a DataFrame named df with a column named 'Size'
# Replace df['Size'] with your actual DataFrame column if different


df['Size'] = df['Size'].str.replace(r'sqft|sf', '', regex=True)

# Display the modified 'Size' column
print(df['Size'])


0        1335.035
1        1875.075
2        1513.013
3        7200.000
4        3600.000
           ...   
29027     915.015
29028    1544.044
29029     650.050
29030    3973.073
29031    1313.013
Name: Size, Length: 29032, dtype: object


In [334]:
df['Size'].value_counts()

Size
1100.000     392
48.575       320
950.050      294
1200.000     265
850.050      235
            ... 
13558.058      1
6135.035       1
3434.034       1
47.560         1
2132.032       1
Name: count, Length: 3311, dtype: int64

In [335]:
df["Size"]

0        1335.035
1        1875.075
2        1513.013
3        7200.000
4        3600.000
           ...   
29027     915.015
29028    1544.044
29029     650.050
29030    3973.073
29031    1313.013
Name: Size, Length: 29032, dtype: object

In [336]:
# Function to filter out non-float values


import re
def filter_non_float_values(sizes):
    non_float_values = []
    for size in sizes:
        if isinstance(size, str):
            # Using regular expression to exclude float values
            matches = re.findall(r'\b\d+\.\d+\b', size)
            if not matches:
                non_float_values.append(size)
        else:
            non_float_values.append(size)
    return non_float_values

# Find unique non-float values in the 'Size' column
unique_sizes = filter_non_float_values(df['Size'].unique())

# Display unique non-float values
for index, size in enumerate(unique_sizes):
    print(f"Row index {index}: {size}")


Row index 0: nan
Row index 1: 51.5ft
Row index 2: 531.6666666666666f)
Row index 3: 1516.6666666666667f)
Row index 4: 45.0es
Row index 5: 00ur
Row index 6: 1017.5ft
Row index 7: nilil
Row index 8: Malaysiaia
Row index 9: CornerUnitit
Row index 10: unknownwn
Row index 11: 46.0ft
Row index 12: WPWP
Row index 13: NANA
Row index 14: 82.0es
Row index 15: 27.0xx
Row index 16: 1.0re
Row index 17: 0an
Row index 18: 30.0ft
Row index 19: 26.0ce
Row index 20: cornerer
Row index 21: 51.0ft
Row index 22: 823.5ft


In [337]:

# Define regex pattern to extract numeric values
pattern = r'(\d*\.?\d+)'

# Apply regex operation to extract numeric values
df['Size'] = df['Size'].str.extract(pattern, expand=False).astype(float)

df

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,original_size
0,KLCC,"RM 1,250,000",2+1,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,1335sqft
1,Dutamas,"RM 1,030,000",3,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,1875sqft
2,Bukit Jalil,"RM 900,000",4+1,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,1513sqft
3,Taman Tun Dr Ismail,"RM 5,350,000",4+2,5,4,Bungalow,7200.000,Partly Furnished,Land area,7200sqft
4,Taman Tun Dr Ismail,"RM 2,600,000",5,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,3600sqft
...,...,...,...,...,...,...,...,...,...,...
29027,Seputeh,"RM 750,000",3,2,1,Condominium,915.015,Partly Furnished,Built-up,915sqft
29028,KL Sentral,"RM 1,400,000",3+1,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,1544sqft
29029,KL Eco City,"RM 880,000",1,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,650sqft
29030,Sri Hartamas,"RM 2,700,000",4+2,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,3973sqft


In [338]:
df["Size"].astype("float")

0        1335.035
1        1875.075
2        1513.013
3        7200.000
4        3600.000
           ...   
29027     915.015
29028    1544.044
29029     650.050
29030    3973.073
29031    1313.013
Name: Size, Length: 29032, dtype: float64

In [339]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29032 entries, 0 to 29031
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       29032 non-null  object 
 1   Price          29032 non-null  object 
 2   Rooms          29032 non-null  object 
 3   Bathrooms      29032 non-null  int64  
 4   Car Parks      29032 non-null  int64  
 5   Property Type  29032 non-null  object 
 6   Size           28990 non-null  float64
 7   Furnishing     29032 non-null  object 
 8   Type           29032 non-null  object 
 9   original_size  29032 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 2.2+ MB


In [340]:
df

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,original_size
0,KLCC,"RM 1,250,000",2+1,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,1335sqft
1,Dutamas,"RM 1,030,000",3,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,1875sqft
2,Bukit Jalil,"RM 900,000",4+1,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,1513sqft
3,Taman Tun Dr Ismail,"RM 5,350,000",4+2,5,4,Bungalow,7200.000,Partly Furnished,Land area,7200sqft
4,Taman Tun Dr Ismail,"RM 2,600,000",5,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,3600sqft
...,...,...,...,...,...,...,...,...,...,...
29027,Seputeh,"RM 750,000",3,2,1,Condominium,915.015,Partly Furnished,Built-up,915sqft
29028,KL Sentral,"RM 1,400,000",3+1,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,1544sqft
29029,KL Eco City,"RM 880,000",1,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,650sqft
29030,Sri Hartamas,"RM 2,700,000",4+2,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,3973sqft


In [341]:
df.drop(columns= "original_size", inplace =True)


In [342]:
df

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type
0,KLCC,"RM 1,250,000",2+1,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up
1,Dutamas,"RM 1,030,000",3,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up
2,Bukit Jalil,"RM 900,000",4+1,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up
3,Taman Tun Dr Ismail,"RM 5,350,000",4+2,5,4,Bungalow,7200.000,Partly Furnished,Land area
4,Taman Tun Dr Ismail,"RM 2,600,000",5,4,4,Semi-detached House,3600.000,Partly Furnished,Land area
...,...,...,...,...,...,...,...,...,...
29027,Seputeh,"RM 750,000",3,2,1,Condominium,915.015,Partly Furnished,Built-up
29028,KL Sentral,"RM 1,400,000",3+1,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area
29029,KL Eco City,"RM 880,000",1,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up
29030,Sri Hartamas,"RM 2,700,000",4+2,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up


In [343]:
data=df

In [344]:
data

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type
0,KLCC,"RM 1,250,000",2+1,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up
1,Dutamas,"RM 1,030,000",3,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up
2,Bukit Jalil,"RM 900,000",4+1,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up
3,Taman Tun Dr Ismail,"RM 5,350,000",4+2,5,4,Bungalow,7200.000,Partly Furnished,Land area
4,Taman Tun Dr Ismail,"RM 2,600,000",5,4,4,Semi-detached House,3600.000,Partly Furnished,Land area
...,...,...,...,...,...,...,...,...,...
29027,Seputeh,"RM 750,000",3,2,1,Condominium,915.015,Partly Furnished,Built-up
29028,KL Sentral,"RM 1,400,000",3+1,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area
29029,KL Eco City,"RM 880,000",1,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up
29030,Sri Hartamas,"RM 2,700,000",4+2,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up


In [345]:
#Cleaning Price column
data['Price'] = data['Price'].str.replace(r'\D+', '', regex=True).astype(int)

data

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type
0,KLCC,1250000,2+1,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up
1,Dutamas,1030000,3,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up
2,Bukit Jalil,900000,4+1,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up
3,Taman Tun Dr Ismail,5350000,4+2,5,4,Bungalow,7200.000,Partly Furnished,Land area
4,Taman Tun Dr Ismail,2600000,5,4,4,Semi-detached House,3600.000,Partly Furnished,Land area
...,...,...,...,...,...,...,...,...,...
29027,Seputeh,750000,3,2,1,Condominium,915.015,Partly Furnished,Built-up
29028,KL Sentral,1400000,3+1,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area
29029,KL Eco City,880000,1,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up
29030,Sri Hartamas,2700000,4+2,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up


In [346]:
# Splitting the Rooms column and creating two new columns, no of bed room, servant rooms
data[['No_of_Bedrooms', 'No_of_servant_rooms']] = data['Rooms'].str.split('+', expand=True)
data.drop(columns="Rooms", inplace=True)


In [347]:
data['No_of_servant_rooms'].isna().sum()

17243

In [348]:
data['No_of_servant_rooms'].fillna(value=0, inplace=True)

In [349]:
data

,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0
...,...,...,...,...,...,...,...,...,...,...
29027,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0
29028,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1
29029,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0
29030,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2


In [350]:
# Convert 'Bathrooms' column to integer
data['Bathrooms'] = data['Bathrooms'].astype(int)
data

,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0
...,...,...,...,...,...,...,...,...,...,...
29027,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0
29028,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1
29029,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0
29030,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2


In [351]:
# Convert 'Car Parks' column to integer
data['Car Parks'] = data['Car Parks'].astype(int)
data

,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0
...,...,...,...,...,...,...,...,...,...,...
29027,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0
29028,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1
29029,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0
29030,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2


In [352]:
data

,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0
...,...,...,...,...,...,...,...,...,...,...
29027,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0
29028,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1
29029,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0
29030,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2


In [353]:
data.isna().sum()

Location                0
Price                   0
Bathrooms               0
Car Parks               0
Property Type           0
Size                   42
Furnishing              0
Type                    0
No_of_Bedrooms          0
No_of_servant_rooms     0
dtype: int64

In [354]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29032 entries, 0 to 29031
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             29032 non-null  object 
 1   Price                29032 non-null  int32  
 2   Bathrooms            29032 non-null  int32  
 3   Car Parks            29032 non-null  int32  
 4   Property Type        29032 non-null  object 
 5   Size                 28990 non-null  float64
 6   Furnishing           29032 non-null  object 
 7   Type                 29032 non-null  object 
 8   No_of_Bedrooms       29032 non-null  object 
 9   No_of_servant_rooms  29032 non-null  object 
dtypes: float64(1), int32(3), object(6)
memory usage: 1.9+ MB


In [355]:
data= data.dropna()
data.reset_index(drop=True, inplace=True) 
data

,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0
...,...,...,...,...,...,...,...,...,...,...
28985,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0
28986,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1
28987,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0
28988,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2


data['size'] = data['size'].astype(int)

In [356]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28990 entries, 0 to 28989
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             28990 non-null  object 
 1   Price                28990 non-null  int32  
 2   Bathrooms            28990 non-null  int32  
 3   Car Parks            28990 non-null  int32  
 4   Property Type        28990 non-null  object 
 5   Size                 28990 non-null  float64
 6   Furnishing           28990 non-null  object 
 7   Type                 28990 non-null  object 
 8   No_of_Bedrooms       28990 non-null  object 
 9   No_of_servant_rooms  28990 non-null  object 
dtypes: float64(1), int32(3), object(6)
memory usage: 1.9+ MB


In [357]:
# Use regex to remove non-digit characters and keep only numbers
data['No_of_servant_rooms'] = data['No_of_servant_rooms'].str.replace(r'\D+', '', regex=True)

# Replace empty strings or NaN values with a placeholder value (e.g., '0')
data['No_of_servant_rooms'].replace('', '0', inplace=True)
data['No_of_servant_rooms'].fillna('0', inplace=True)

# Convert the column to integer type
data['No_of_servant_rooms'] = data['No_of_servant_rooms'].astype(int)


C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\4288363492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['No_of_servant_rooms'] = data['No_of_servant_rooms'].str.replace(r'\D+', '', regex=True)
C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\4288363492.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['No_of_servant_rooms'].replace('', '0', inplace=True)
C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\4288363492.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

In [358]:
data

,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0
...,...,...,...,...,...,...,...,...,...,...
28985,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0
28986,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1
28987,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0
28988,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2


In [359]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28990 entries, 0 to 28989
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             28990 non-null  object 
 1   Price                28990 non-null  int32  
 2   Bathrooms            28990 non-null  int32  
 3   Car Parks            28990 non-null  int32  
 4   Property Type        28990 non-null  object 
 5   Size                 28990 non-null  float64
 6   Furnishing           28990 non-null  object 
 7   Type                 28990 non-null  object 
 8   No_of_Bedrooms       28990 non-null  object 
 9   No_of_servant_rooms  28990 non-null  int32  
dtypes: float64(1), int32(4), object(5)
memory usage: 1.8+ MB


In [360]:
# Calculate minimum and maximum size values for studio records
min_size_studio = data.loc[data['No_of_Bedrooms'] == 'Studio', 'Size'].min()
max_size_studio = data.loc[data['No_of_Bedrooms'] == 'Studio', 'Size'].max()
mean_size_studio = data.loc[data['No_of_Bedrooms'] == 'Studio', 'Size'].mean()

print("Minimum size for studio records:", min_size_studio)
print("Maximum size for studio records:", max_size_studio)
print("Mean size for studio records:", mean_size_studio)

Minimum size for studio records: 250.05
Maximum size for studio records: 243900.0
Mean size for studio records: 1423.78175


In [361]:
data['No_of_Bedrooms'].value_counts()

No_of_Bedrooms
3           13943
4            4983
2            4497
5            2097
1            1992
6             752
Studio        312
7             255
8              97
9              22
10             20
12              5
20 Above        4
13              4
11              3
14              2
15              1
16              1
Name: count, dtype: int64

In [362]:
# Create a new column 'studio_apartment'
data['studio_apartment'] = data['No_of_Bedrooms'].apply(lambda x: 1 if x == 'Studio' else 0)

C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\681213311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['studio_apartment'] = data['No_of_Bedrooms'].apply(lambda x: 1 if x == 'Studio' else 0)


In [363]:
# Replace 'studio' values in 'No_of_Bedrooms' column with 0
data['No_of_Bedrooms'].replace({'Studio': 0, '20 Above': 21}, inplace=True)

data

C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\2864032243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['No_of_Bedrooms'].replace({'Studio': 0, '20 Above': 21}, inplace=True)


,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms,studio_apartment
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1,0
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1,0
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2,0
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...
28985,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0,0
28986,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1,0
28987,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0,0
28988,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2,0


In [364]:
data.studio_apartment.value_counts()

studio_apartment
0    28678
1      312
Name: count, dtype: int64

In [365]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28990 entries, 0 to 28989
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             28990 non-null  object 
 1   Price                28990 non-null  int32  
 2   Bathrooms            28990 non-null  int32  
 3   Car Parks            28990 non-null  int32  
 4   Property Type        28990 non-null  object 
 5   Size                 28990 non-null  float64
 6   Furnishing           28990 non-null  object 
 7   Type                 28990 non-null  object 
 8   No_of_Bedrooms       28990 non-null  object 
 9   No_of_servant_rooms  28990 non-null  int32  
 10  studio_apartment     28990 non-null  int64  
dtypes: float64(1), int32(4), int64(1), object(5)
memory usage: 2.0+ MB


In [366]:
data['No_of_Bedrooms']=data['No_of_Bedrooms'].astype(int)

C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\1804879422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['No_of_Bedrooms']=data['No_of_Bedrooms'].astype(int)


In [367]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28990 entries, 0 to 28989
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             28990 non-null  object 
 1   Price                28990 non-null  int32  
 2   Bathrooms            28990 non-null  int32  
 3   Car Parks            28990 non-null  int32  
 4   Property Type        28990 non-null  object 
 5   Size                 28990 non-null  float64
 6   Furnishing           28990 non-null  object 
 7   Type                 28990 non-null  object 
 8   No_of_Bedrooms       28990 non-null  int32  
 9   No_of_servant_rooms  28990 non-null  int32  
 10  studio_apartment     28990 non-null  int64  
dtypes: float64(1), int32(5), int64(1), object(4)
memory usage: 1.9+ MB


In [368]:
data

,Location,Price,Bathrooms,Car Parks,Property Type,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms,studio_apartment
0,KLCC,1250000,3,2,Serviced Residence,1335.035,Fully Furnished,Built-up,2,1,0
1,Dutamas,1030000,4,2,Condominium (Corner),1875.075,Partly Furnished,Built-up,3,0,0
2,Bukit Jalil,900000,3,2,Condominium (Corner),1513.013,Partly Furnished,Built-up,4,1,0
3,Taman Tun Dr Ismail,5350000,5,4,Bungalow,7200.000,Partly Furnished,Land area,4,2,0
4,Taman Tun Dr Ismail,2600000,4,4,Semi-detached House,3600.000,Partly Furnished,Land area,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...
28985,Seputeh,750000,2,1,Condominium,915.015,Partly Furnished,Built-up,3,0,0
28986,KL Sentral,1400000,3,2,Condominium (Corner),1544.044,Fully Furnished,Land area,3,1,0
28987,KL Eco City,880000,1,1,Condominium (Corner),650.050,Partly Furnished,Built-up,1,0,0
28988,Sri Hartamas,2700000,6,3,Condominium (Corner),3973.073,Partly Furnished,Built-up,4,2,0


In [369]:
def simplify_property_type(prop_type):
    super_types = [
        "Terrace/Link House", "Serviced Residence", "Condominium",
        "Semi-detached House", "Bungalow", "Apartment", "Townhouse",
        "Flat", "Residential Land", "Cluster House"
    ]
    for super_type in super_types:
        if re.search(super_type, prop_type, flags=re.IGNORECASE) is not None:
            return super_type
    return prop_type

# Assuming properties is your DataFrame containing the "Property Type" column
data["Property Type Supergroup"] = data["Property Type"].apply(simplify_property_type)
data.drop(columns='Property Type', inplace= True)

# Display value counts for 'Property Type Supergroup'
print(data["Property Type Supergroup"].value_counts(dropna=False))

Property Type Supergroup
Condominium            14503
Serviced Residence      7611
Terrace/Link House      3183
Bungalow                1412
Apartment               1038
Semi-detached House      828
Townhouse                284
Flat                     102
Residential Land          23
Cluster House              6
Name: count, dtype: int64


C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\1647539945.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Property Type Supergroup"] = data["Property Type"].apply(simplify_property_type)
C:\Users\prabh\AppData\Local\Temp\ipykernel_15832\1647539945.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns='Property Type', inplace= True)


In [370]:
data

,Location,Price,Bathrooms,Car Parks,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms,studio_apartment,Property Type Supergroup
0,KLCC,1250000,3,2,1335.035,Fully Furnished,Built-up,2,1,0,Serviced Residence
1,Dutamas,1030000,4,2,1875.075,Partly Furnished,Built-up,3,0,0,Condominium
2,Bukit Jalil,900000,3,2,1513.013,Partly Furnished,Built-up,4,1,0,Condominium
3,Taman Tun Dr Ismail,5350000,5,4,7200.000,Partly Furnished,Land area,4,2,0,Bungalow
4,Taman Tun Dr Ismail,2600000,4,4,3600.000,Partly Furnished,Land area,5,0,0,Semi-detached House
...,...,...,...,...,...,...,...,...,...,...,...
28985,Seputeh,750000,2,1,915.015,Partly Furnished,Built-up,3,0,0,Condominium
28986,KL Sentral,1400000,3,2,1544.044,Fully Furnished,Land area,3,1,0,Condominium
28987,KL Eco City,880000,1,1,650.050,Partly Furnished,Built-up,1,0,0,Condominium
28988,Sri Hartamas,2700000,6,3,3973.073,Partly Furnished,Built-up,4,2,0,Condominium


In [371]:
data.to_csv('KL_clean_data.csv',index=False)

In [372]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28990 entries, 0 to 28989
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Location                  28990 non-null  object 
 1   Price                     28990 non-null  int32  
 2   Bathrooms                 28990 non-null  int32  
 3   Car Parks                 28990 non-null  int32  
 4   Size                      28990 non-null  float64
 5   Furnishing                28990 non-null  object 
 6   Type                      28990 non-null  object 
 7   No_of_Bedrooms            28990 non-null  int32  
 8   No_of_servant_rooms       28990 non-null  int32  
 9   studio_apartment          28990 non-null  int64  
 10  Property Type Supergroup  28990 non-null  object 
dtypes: float64(1), int32(5), int64(1), object(4)
memory usage: 1.9+ MB


In [373]:
data

,Location,Price,Bathrooms,Car Parks,Size,Furnishing,Type,No_of_Bedrooms,No_of_servant_rooms,studio_apartment,Property Type Supergroup
0,KLCC,1250000,3,2,1335.035,Fully Furnished,Built-up,2,1,0,Serviced Residence
1,Dutamas,1030000,4,2,1875.075,Partly Furnished,Built-up,3,0,0,Condominium
2,Bukit Jalil,900000,3,2,1513.013,Partly Furnished,Built-up,4,1,0,Condominium
3,Taman Tun Dr Ismail,5350000,5,4,7200.000,Partly Furnished,Land area,4,2,0,Bungalow
4,Taman Tun Dr Ismail,2600000,4,4,3600.000,Partly Furnished,Land area,5,0,0,Semi-detached House
...,...,...,...,...,...,...,...,...,...,...,...
28985,Seputeh,750000,2,1,915.015,Partly Furnished,Built-up,3,0,0,Condominium
28986,KL Sentral,1400000,3,2,1544.044,Fully Furnished,Land area,3,1,0,Condominium
28987,KL Eco City,880000,1,1,650.050,Partly Furnished,Built-up,1,0,0,Condominium
28988,Sri Hartamas,2700000,6,3,3973.073,Partly Furnished,Built-up,4,2,0,Condominium
